In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
from traceback import format_exc
# from pprint import pprint
import the_networks_of_war_python_functions

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
csv_directory = '/Users/the_networks_of_war/data_sources/csvs/'
pickle_directory = '/Users/the_networks_of_war/data_sources/pickles/'

# Descriptive Statistics for Each Country by Year
### Note: Applies to states/countries only.
### This will be joined to the participants of each war

### First, defining the participants by year to exclude any countries/years that will not be needed later on.

In [4]:
## unioning start and end years since that's how participant data will be joined in step 3
initial_part_df_1 = pd.read_pickle(pickle_directory + 'initial_part_df.pkl')[['c_code', 'start_year']]
initial_part_df_1.rename({'start_year': 'year'}, axis=1, inplace=True)
## year cannot be null
initial_part_df_1 = deepcopy(initial_part_df_1[initial_part_df_1['year'].isnull()==False])
initial_part_df_2 = pd.read_pickle(pickle_directory + 'initial_part_df.pkl')[['c_code', 'end_year']]
initial_part_df_2.rename({'end_year': 'year'}, axis=1, inplace=True)
## year cannot be null
initial_part_df_2 = deepcopy(initial_part_df_2[initial_part_df_2['year'].isnull()==False])

initial_part_df = deepcopy(pd.concat([initial_part_df_1, initial_part_df_2], sort=True, ignore_index=True).reset_index(drop=True))

initial_part_columns = ['c_code', 'year']
initial_part_df.drop_duplicates(subset=initial_part_columns, keep='first', inplace = True)
print('Total Yearly Participants in Initial Data: {}'.format(format(len(initial_part_df), ',d')))

Total Yearly Participants in Initial Data: 1,557


In [5]:
part_df_1 = pd.read_csv(csv_directory + 'alliance_v4.1_by_member_yearly.csv', encoding = 'latin-1')
part_df_1.rename({'ccode': 'c_code',
                  'defense': 'defense_alliances',
                  'entente': 'entente_alliances',
                  'neutrality': 'neutrality_alliances',
                  'ss_type': 'alliances'}, axis = 1, inplace = True)
## inner join to only include participants found in participants war data
## this will limit runtime significantly
part_df_1 = deepcopy(pd.merge(initial_part_df, part_df_1, how='inner', on=['c_code', 'year']))
    
part_df_1['defense_alliances'] = part_df_1['defense_alliances'].astype(float)
part_df_1['entente_alliances'] = part_df_1['entente_alliances'].astype(float)
part_df_1['neutrality_alliances'] = part_df_1['neutrality_alliances'].astype(float)

aggregations = {
    'alliances': 'count',
    'defense_alliances': 'sum',
    'entente_alliances': 'sum',
    'neutrality_alliances': 'sum',
    }
part_df_1 = deepcopy(part_df_1.groupby(['c_code', 'year']).agg(aggregations).reset_index())
## removing any duplicates that may have occured
part_df_1.drop_duplicates(subset=['c_code', 'year'], keep='first', inplace = True)

In [6]:
## dyadic trade data that will need to be adjusted to be non-dyadic (by country, by year)
part_df_2_1 = pd.read_csv(csv_directory + 'Dyadic_COW_4.0.csv', encoding='utf8')
part_df_2_1.rename({'ccode1': 'c_code_a',
                    'ccode2': 'c_code_b',
                    'flow2': 'money_flow_in_a',
                    ## money flow out
                    'flow1': 'money_flow_in_b'}, axis = 1, inplace = True)

## need to union to take summations but won't need to dedupe because there are no duplicates between a and b.
# this means a can be summed on its own when it's combined with b.
part_df_2_1 = deepcopy(the_networks_of_war_python_functions.union_opposite_columns(part_df_2_1))
part_df_2_1.rename({'money_flow_in_a': 'money_flow_in',
                    'money_flow_in_b': 'money_flow_out'}, axis = 1, inplace = True)

aggregations = {'money_flow_in': 'sum',
                'money_flow_out': 'sum'}
part_df_2_1 = part_df_2_1.groupby(['c_code_a', 'year']).agg(aggregations).reset_index()
part_df_2_1.rename({'c_code_a':'c_code'}, axis = 1, inplace = True)

part_df_2_2 = pd.read_csv(csv_directory + 'National_COW_4.0.csv', encoding = 'latin-1')
part_df_2_2.rename({'ccode': 'c_code'}, axis = 1, inplace = True)
part_df_2_2 = deepcopy(part_df_2_2[['c_code',
                                    'year',
                                    'imports',
                                    'exports']])

part_df_2 = deepcopy(pd.merge(part_df_2_1, part_df_2_2, how='outer', on=['c_code', 'year']))

## inner join to only include participants found in participants war data
## this will limit runtime significantly
part_df_2 = deepcopy(pd.merge(initial_part_df, part_df_2, how='inner', on=['c_code', 'year']))
## removing any duplicates that may have occured
## need to count how many of these there are
part_df_2.drop_duplicates(subset=['c_code', 'year'], keep='first', inplace = True)

In [7]:
# trade_df.head(3)

In [8]:
part_df_3 = pd.read_csv(csv_directory + 'NMC_5_0-wsupplementary.csv', encoding='latin-1')
## including columns that need to be included later on (that don't need name changes)
part_df_3_renaming = {'ccode': 'c_code',
                      'milex': 'military_expenditure',
                      'milper': 'military_personnel',
                      'irst': 'iron_steel_production',
                      'pec': 'prim_energy_consumption',
                      'tpop': 'total_population',
                      'upop': 'urban_population',
                      'upopgrowth': 'urban_pop_growth_rate',
                      'cinc': 'cinc_score',
                      'year': 'year'}
part_df_3.rename(part_df_3_renaming, axis=1, inplace=True)

part_df_3 = part_df_3.sort_values(by='year', ascending=True).reset_index(drop=True)
part_df_3 = deepcopy(part_df_3[list(part_df_3_renaming.values())])

## inner join to only include participants found in participants war data
## this will limit runtime significantly
part_df_3 = deepcopy(pd.merge(initial_part_df, part_df_3, how='inner', on=['c_code', 'year']))
## removing any duplicates that may have occured
part_df_3.drop_duplicates(subset=['c_code', 'year'], keep='first', inplace=True)

In [9]:
# ## not sure that this one is worth the trouble because ccodes are not included
# part_df_4 = pd.read_csv(csv_directory + 'co-emissions-per-capita.csv', encoding = 'latin-1')

In [10]:
descriptive_df_1 = deepcopy(pd.merge(part_df_1, part_df_2, how='outer', on=['c_code', 'year']))
descriptive_df_1 = deepcopy(pd.merge(descriptive_df_1, part_df_3, how='outer', on=['c_code', 'year']))
descriptive_df_1['year'] = descriptive_df_1['year'].astype(float)

In [11]:
print('Total Rows of Descriptive/Yearly Participant Data: {}'.format(format(len(descriptive_df_1), ',d')))
pickle_directory = '/Users/the_networks_of_war/data_sources/pickles/'
descriptive_df_1.to_pickle(pickle_directory + 'participant_descriptive_df.pkl')

Total Rows of Descriptive/Yearly Participant Data: 1,360


# Descriptive Statistics for Each Dyad by Year
### Note: Applies to states/countries only.
### This will be joined to the dyadic pairs for each war

### First, defining the dyads by year to exclude any countries/years that will not be needed later on.

In [12]:
initial_dyad_columns = ['c_code_a', 'c_code_b', 'year']
initial_dyad_df = pd.read_pickle(pickle_directory + 'initial_dyad_df.pkl')[initial_dyad_columns]

## must be a state (non-state participants will not be joined below)
## this will remove -8 and -9 participants
## must have a valid year since all joins below are by dyad yearly
initial_dyad_df = deepcopy(initial_dyad_df[(initial_dyad_df['c_code_a']>0) & (initial_dyad_df['c_code_b']>0) & (initial_dyad_df['year'].isnull()==False)])
initial_dyad_df.drop_duplicates(subset=initial_dyad_columns, keep='first', inplace=True)
initial_dyad_df = deepcopy(initial_dyad_df.reset_index(drop=True))

print('Total Dyadic Years in Initial Data: {}'.format(format(int(len(initial_dyad_df)/2), ',d')))

Total Dyadic Years in Initial Data: 776


### Correlates of War Descriptive Data

In [13]:
print('Counting Total Dyadic Year Combinations by Descriptive Field\n')
initial_descriptive_columns = deepcopy(set(list(initial_dyad_df.columns)))

## lot's to use in this dataset so I'll start with the basics
data_source = csv_directory + 'tc2018.csv'
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, None, 'gainer', 'loser', 'year', 'territory_exchange'))

## contiguity dataframe for states of colonial dependencies
data_source = csv_directory + 'contcold.csv'
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, data_source, None, None, 'statelno', 'statehno', 'year', 'colonial_contiguity'))

data_source = csv_directory + 'contdird.csv'
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, data_source, None, None, 'state1no', 'state2no', 'year', 'contiguity'))

data_source = csv_directory + 'alliance_v4.1_by_dyad_yearly.csv'
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, data_source, None, None, 'ccode1', 'ccode2', 'year', 'alliance'))

data_source = csv_directory + 'DCAD-v1.0-dyadic.csv'
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, data_source, None, None, 'ccode1', 'ccode2', 'year', 'defense_cooperation_agreements'))

## must have at least one joined by both states in order to be included
data_source = csv_directory + 'dyadic_formatv3.csv'
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, data_source, None, None, 'ccode1', 'ccode2', 'year', 'inter_governmental_organizations'))

## this one needs to be filled since its only 5 years
data_source = csv_directory + 'Diplomatic_Exchange_2006v1.csv'
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, data_source, None, None, 'ccode1', 'ccode2', 'year', 'diplomatic_exchange'))

data_source = csv_directory + 'Dyadic_COW_4.0.csv'
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, data_source, None, None, 'ccode1', 'ccode2', 'year', 'trade_relations'))

the_networks_of_war_python_functions.print_new_fields(descriptive_df_2, initial_descriptive_columns, None)

Counting Total Dyadic Year Combinations by Descriptive Field

 inter_governmental_organizations  776
                  trade_relations  684
                       contiguity  348
              diplomatic_exchange  116
              colonial_contiguity  101
   defense_cooperation_agreements   76
                         alliance   64
               territory_exchange   61


### Non-Correlates of War Descriptive Data

### Setting up processing of 'ddrevisited_data_v1'

In [14]:
dd_df_1 = pd.read_csv(csv_directory + 'ddrevisited_data_v1.csv', encoding = 'latin-1')
## including columns that need to be included later on (that don't need name changes)
dd_df_renaming = {'cowcode': 'c_code',
                  'cid': 'alternate_country_code',
                  'emil': 'military_leader',
                  'royal': 'royal_leader',
                  'comm': 'communist_leader',
                  'democracy': 'democratic_regime',
                  'collect': 'collective_leadership',
                  'regime': 'regime_type',
                  'incumb': 'incumbent_type',
                  'exselec': 'election_type',
                  'legselec': 'legislature_type_1',
                  'closed': 'legislature_type_2',
                  'lparty': 'legislature_party_status',
                  'dejure': 'party_legal_status',
                  'defacto': 'party_existance_1',
                  'defacto2': 'party_existance_2',
                  'year': 'year'}
dd_df_1.rename(dd_df_renaming, axis=1, inplace=True)
dd_df_1 = deepcopy(dd_df_1[list(dd_df_renaming.values())])

## replacing cowcode with cowcode2 for second version
del dd_df_renaming['cowcode']
dd_df_renaming['cowcode2'] = 'c_code'

dd_df_2 = pd.read_csv(csv_directory + 'ddrevisited_data_v1.csv', encoding = 'latin-1')
dd_df_2.rename(dd_df_renaming, axis=1, inplace=True)
dd_df_2 = deepcopy(dd_df_2[list(dd_df_renaming.values())])

## unioning the two ccodes above so they can both be represented
## this will also allow for substates to be joined to the larger states
## this will need to be recognized later on to prevent from saying same leadership when it's the same leader
dd_df = deepcopy(pd.concat([dd_df_1, dd_df_2], sort=True, ignore_index=True))

## removing duplicates from concat
dd_df.drop_duplicates(subset=list(dd_df_renaming.values()), keep='first', inplace = True)

## transforming this into a dyadic dataset (each country joined to each country)
dd_df = deepcopy(pd.merge(dd_df, dd_df, how='left', on=['year']))
    
for column in dd_df.columns:
    if column[-2:]=='_x':
        dd_df.rename({column: column[:-2] + '_a'}, axis=1, inplace=True)
    elif column[-2:]=='_y':
        dd_df.rename({column: column[:-2] + '_b'}, axis=1, inplace=True)
    else:
        pass

## inner join to only include dyads found in dyadic war data
## this will limit runtime significantly
dd_df = deepcopy(pd.merge(initial_dyad_df, dd_df, how='inner', on=['c_code_a', 'c_code_b', 'year']))

## ensuring that c_code_a and c_code_b do not match, and cowcode and cowcode2 (overlapping states) are not joined to each other.
dd_df = deepcopy(dd_df[(dd_df['c_code_a']!=dd_df['c_code_b']) & (dd_df['alternate_country_code_a']!=dd_df['alternate_country_code_b'])].reset_index(drop=True))
dd_df.drop(['alternate_country_code_a',
            'alternate_country_code_b'], axis=1, inplace=True)

In [15]:
print('Counting Total Dyadic Year Combinations by Descriptive Field\n')
initial_descriptive_columns = deepcopy(set(list(descriptive_df_2.columns)))

conditional_statement = (dd_df['military_leader_a']==dd_df['military_leader_b']) & (dd_df['communist_leader_a']==dd_df['communist_leader_b']) & (dd_df['royal_leader_a']==dd_df['royal_leader_b']) & (dd_df['democratic_regime_a']==dd_df['democratic_regime_b'])
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'same_leader_type'))

conditional_statement = (dd_df['military_leader_a']==1) & (dd_df['military_leader_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'military_leaders'))

conditional_statement = (dd_df['communist_leader_a']==1) & (dd_df['communist_leader_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'communist_leaders'))

conditional_statement = (dd_df['royal_leader_a']==1) & (dd_df['royal_leader_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'royal_leaders'))

conditional_statement = (dd_df['incumbent_type_a']==1) & (dd_df['incumbent_type_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'democratic_incumbent'))

conditional_statement = (dd_df['incumbent_type_a']==2) & (dd_df['incumbent_type_b']==2)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'unconstitutional_incumbent'))

conditional_statement = ((dd_df['incumbent_type_a']==1) | (dd_df['democratic_regime_a']==1) | (dd_df['regime_type_a']==0) | (dd_df['regime_type_a']==1) | (dd_df['regime_type_a']==2)) & ((dd_df['incumbent_type_b']==1) |(dd_df['democratic_regime_b']==1) | (dd_df['regime_type_b']==0) | (dd_df['regime_type_b']==1) | (dd_df['regime_type_b']==2))
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'democratic_regimes'))

conditional_statement = ((dd_df['regime_type_a']==3) | (dd_df['regime_type_a']==4) | (dd_df['regime_type_a']==5)) & ((dd_df['regime_type_b']==3) | (dd_df['regime_type_b']==4) | (dd_df['regime_type_b']==5))
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'dictatorships'))

conditional_statement = (dd_df['collective_leadership_a']==1) & (dd_df['collective_leadership_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'collective_leaderships'))

conditional_statement = (dd_df['election_type_a']==1) & (dd_df['election_type_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'direct_election'))

conditional_statement = (dd_df['election_type_a']==2) & (dd_df['election_type_b']==2)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'indirect_election'))

conditional_statement = (dd_df['election_type_a']==3) & (dd_df['election_type_b']==3)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'non_elected_leaders'))

conditional_statement = (dd_df['legislature_type_1_a']==0) & (dd_df['legislature_type_1_b']==0)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'no_legislature'))

conditional_statement = (dd_df['legislature_type_1_a']==1) & (dd_df['legislature_type_1_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'non_elective_legislature'))

conditional_statement = (dd_df['legislature_type_1_a']==2) & (dd_df['legislature_type_1_b']==2)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'nelective_legislature'))

conditional_statement = (dd_df['legislature_party_status_a']==0) & (dd_df['legislature_party_status_b']==0)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'no_partisan_legislature_legal'))

conditional_statement = (dd_df['legislature_party_status_a']==1) & (dd_df['legislature_party_status_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'no_non_regime_legislature_parties_legal'))

conditional_statement = (dd_df['legislature_party_status_a']==2) & (dd_df['legislature_party_status_b']==2)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'multi_party_legislature_legal'))

conditional_statement = (dd_df['party_legal_status_a']==0) & (dd_df['party_legal_status_b']==0)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'all_parties_illegal'))

conditional_statement = (dd_df['party_legal_status_a']==1) & (dd_df['party_legal_status_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'single_party_state_exists'))

conditional_statement = (dd_df['party_legal_status_a']==2) & (dd_df['party_legal_status_b']==2)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'multi_party_state_exists'))

conditional_statement = (dd_df['party_existance_1_a']==0) & (dd_df['party_existance_1_b']==0)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'no_parties_exist'))

conditional_statement = (dd_df['party_existance_1_a']==1) & (dd_df['party_existance_1_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'one_party_exists'))

conditional_statement = (dd_df['party_existance_2_a']==1) & (dd_df['party_existance_2_b']==1)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', dd_df, conditional_statement, 'c_code_a', 'c_code_b', 'year', 'no_non_regime_parties_exist'))

the_networks_of_war_python_functions.print_new_fields(descriptive_df_2, initial_descriptive_columns, None)

Counting Total Dyadic Year Combinations by Descriptive Field

                   nelective_legislature  179
                multi_party_state_exists  169
                           dictatorships  112
                     non_elected_leaders   67
                        same_leader_type   36
           multi_party_legislature_legal   33
             no_non_regime_parties_exist   32
                        military_leaders   24
           no_partisan_legislature_legal   22
                       indirect_election   19
                      democratic_regimes   12
                          no_legislature   12
                         direct_election   11
 no_non_regime_legislature_parties_legal   11
                        one_party_exists    9
                       communist_leaders    6
                non_elective_legislature    3
                     all_parties_illegal    2
               single_party_state_exists    2
                        no_parties_exist    1
                  

### Other Non-COW Data Sources

In [16]:
print('Counting Total Dyadic Year Combinations by Descriptive Field\n')
initial_descriptive_columns = deepcopy(set(list(descriptive_df_2.columns)))

# Alliance Treaty Obligations and Provisions (ATOP)
data_source = csv_directory + 'atop4_01ddyr.csv'
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, data_source, None, None, 'stateA', 'stateB', 'year', 'atop'))

# The ICOW multilateral treaties of pacific settlement (MTOPS)
data_source = csv_directory + 'mtopsd150.csv'
mtops_dy_df = pd.read_csv(data_source, encoding='utf8')
mtops_dy_df['total'] = mtops_dy_df['pacsettg'] + mtops_dy_df['pacsettr'] + mtops_dy_df['pacsett'] + mtops_dy_df['tergen'] + mtops_dy_df['terviol'] + mtops_dy_df['tertot']
conditional_statement = (mtops_dy_df['total']>0)
descriptive_df_2 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(descriptive_df_2, 'conditional', mtops_dy_df, conditional_statement, 'state1', 'state2', 'year', 'mtops'))
the_networks_of_war_python_functions.print_new_fields(descriptive_df_2, initial_descriptive_columns, None)

Counting Total Dyadic Year Combinations by Descriptive Field

 mtops  464
  atop   80


### Removing any descriptive dyadic field with 0 records returned.

In [17]:
descriptive_columns = list(descriptive_df_2.drop(['c_code_a', 'c_code_b', 'year'], axis=1).columns)
for column in descriptive_columns:
    if len(descriptive_df_2[descriptive_df_2[column]>0])==0:
        print('Fields Removed: {}'.format(column))
        descriptive_df_2.drop(column, axis=1, inplace=True)

Fields Removed: royal_leaders
Fields Removed: democratic_incumbent
Fields Removed: unconstitutional_incumbent
Fields Removed: collective_leaderships


In [18]:
print('Total Dyadic Years of Descriptive Data: {}'.format(format(int(len(descriptive_df_2)/2), ',d')))
descriptive_df_2.to_pickle(pickle_directory + 'dyadic_descriptive_df.pkl')

Total Dyadic Years of Descriptive Data: 776


In [19]:
# pd.read_csv(csv_directory + 'tc2018.csv', encoding='utf8')


# Process of Territorial Change: The process of territorial change includes six possible procedures:
# 1. Conquest
# 2. Annexation
# 3. Cession
# 4. Secession
# 5. Unification
# 6. Mandated territory

# Portion of the Territory Exchanged: "Portion of unit exchanged" indicates whether part or all of
# the unit being transferred was involved in the exchange. A "0" means that part of the unit was
# transferred, a "1" indicates that the entire unit was involved in the exchange, and a “-9” means
# that this could not be determined.

In [20]:
# data_source = csv_directory + 'ucdp-peace-agreements-191.csv'
# dy_df_19 = pd.read_csv(data_source, encoding='utf8')

# # UCDP Conflict Termination Dataset version 2-2015
# data_source = csv_directory + 'ucdp-term-dyadic-2015.csv'
# dy_df_20 = pd.read_csv(data_source, encoding='utf8')

# # UCDP Battle-Related Deaths Dataset version 20.1
# data_source = csv_directory + 'UCDP Battle-Related Deaths Dataset version 20.1'
# dy_df_21 = pd.read_csv(data_source, encoding='utf8')

# # UCDP Actor Dataset version 20.1
# data_source = csv_directory + 'ucdp-actor-201.csv'
# dy_df_22 = pd.read_csv(data_source, encoding='utf8')

# # UCDP Non-state Conflict Issues and Actors Dataset
# data_source = csv_directory + 'UCDP_NS_IAD.csv'
# dy_df_23 = pd.read_csv(data_source, encoding='utf8')

# # UCDP External Support in Non-state Conflict Dataset
# data_source = csv_directory + 'UCDP External Support in Non-state Conflict Dataset v.1.0.csv'
# dy_df_24 = pd.read_csv(data_source, encoding='utf8')

# # UCDP Managing Intrastate Low-intensity Conflict (MILC) dataset
# data_source = csv_directory + 'milc-10.csv'
# dy_df_25 = pd.read_csv(data_source, encoding='utf8')

# # UCDP Managing Intrastate Conflict (MIC) dataset
# ## more than 2 ids per conflict
# data_source = csv_directory + 'micFINAL.csv'
# dy_df_26 = deepcopy(the_networks_of_war_python_functions.descriptive_dyad_from_source(initial_dyad_df, data_source, None, 'contry_id', 'thirdid1', 'year', 'atop'))

# # 'gwno'
# # 'dyad_id'
# ## only including failed peace agreements
# dy_df_19 = deepcopy(dy_df_19[dy_df_19['ended']==True])[['gwno', 'dyad_id', 'dyad_name', 'actor_id', 'actor_name', 'year', 'duration', 'c_duration']]


In [21]:
# for year in np.arange(1800, 2020):
#     for row in dy_df_7['year']:
#         if len(dy_df_7[dy_df_7['year']==year])== 0:
#             temp_dyad_df = deepcopy(dy_df_7[dy_df_7['year']==year].reset_index())
#             for i, dyad in enumerate(temp_dyad_df['year']):
#                 dyad_df_length = deepcopy(len(dy_df_7))
#                 dy_df_7.loc[dyad_df_length, 'year'] = year
#                 dy_df_7.loc[dyad_df_length, 'c_code_a'] = temp_dyad_df.loc[i, 'c_code_a']
#                 dy_df_7.loc[dyad_df_length, 'c_code_b'] = temp_dyad_df.loc[i, 'c_code_b']
#         else:
#             current_year = year
            
# dy_df_7['diplomatic_exchange'] = 1
# print(len(dy_df_7))